In [1]:
!pip install keras-tuner


     |████████████████████████████████| 71kB 2.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=dd49d14252f67663f92731771015718183bddd46ce19564d94cd81c186821540
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=4c2ac21c0e8f56038996df6eb92d59c2c7a378d48c43c0de5e2a3edf6a3e63d5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [17]:
def build_model(hp):
  model = keras.Sequential([
  keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
      input_shape=(28,28,1)
),
 keras.layers.Conv2D(
     filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
     kernel_size=hp.Choice('conv_2_kernel',values =[3,5 ]),
     activation='relu'
 ),
 keras.layers.Flatten(),
 keras.layers.Dense(
     units=hp.Int('dense_1_ units',min_value=32,max_value=128,step=16),
     activation='relu'
 ),
          keras.layers.Dense(10,activation='softmax')#output layer 
          ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
  loss='sparse_categorical_crossentropy',
   metrics=['accuracy'])
  return model





In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion ")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 21m 22s]
val_accuracy: 0.7771666646003723

Best val_accuracy So Far: 0.7771666646003723
Total elapsed time: 00h 29m 01s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |96                |64                
conv_1_kernel     |5                 |5                 
conv_2_filter     |48                |64                
conv_2_kernel     |3                 |5                 
dense_1_ units    |64                |80                
learning_rate     |0.01              |0.01              

Epoch 1/3
1688/1688 [==============================] - 260s 154ms/step - loss: 2.3376 - accuracy: 0.1006 - val_loss: 2.3035 - val_accuracy: 0.0985
Epoch 2/3
 290/1688 [====>.........................] - ETA: 3:17 - loss: 2.3044 - accuracy: 0.0948

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()